# Imports and Inputs

In [ ]:
import pandas as pd
import numpy as np
from linkml_runtime.dumpers import yaml_dumper
import pypandoc as docgenerator
from typing import Optional

from utils_linkml import(
    validate_data_to_schema, convert_data_to_csv,
    create_python_schema, generate_schema_png, generate_schema_excel,
    generate_schema_documentation,
)

In [ ]:
# raw data upload file path
raw_filepaths = [
    r'C:/Users/johanna.ganglbauer/archive/code/PFAS-SCIEX-Data-Processing-Analysis/example_data_raw/241031_test_data_core.txt',
    r'C:/Users/johanna.ganglbauer/archive/code/PFAS-SCIEX-Data-Processing-Analysis/example_data_raw/241031_test_data_extended.txt',
]

# file paths for IDL and IQL data - not meant to be adopted
idl_2024_filepath = r'C:/Users/johanna.ganglbauer/archive/code/PFAS-SCIEX-Data-Processing-Analysis/example_data_raw/IDL_2024.csv'

# will be used to identify standards by the column 'Component Name'
standard_identifiers = 'IDA|IPS|13C|d-|d3-|d5-|18O'

# Load linkML data model, get illstrations and python data classes

In [ ]:
create_python_schema(yamlschema='chemicals.yaml', pythonschema='chemicals.py')
import chemicals as pyda
generate_schema_png(yamlschema='chemicals.yaml', directory='outputs/chemicals')
generate_schema_excel(yamlschema='chemicals.yaml', excelschema='outputs/chemicals/schema.xlsx')

# Read in and data clean up
- reads in mass spec data results and uses the calibration data to extract relevant information
- reads in excel with associated names and assigns them to PFAS components
- read in IDLs and assigns them to PFAS components

-> combines the data and saves it to the python data classes deduced from linkml

In [ ]:
# Define columns of input which are needed for further processes:
columns_considered = [
    'Sample Name', 'Sample Index', 'Sample Comment', 'Sample Type',
    'Component Name',  'Component Group Name', 'IS Name', 
    'Expected RT',
    'Precursor Mass', 'Fragment Mass',
]

# Load input data files and put them all in one dataframe
data = pd.DataFrame()  # initialize empty data frames
sample_index = 0  # initialize Component Index
for file in raw_filepaths:
    # read in file
    if file[-4:] == '.csv':
        this_data = pd.read_csv(file, delimiter=',', encoding='utf-8', low_memory=False, header=0,)
    elif file[-4:] == '.txt':
        this_data = pd.read_csv(file, delimiter='\t', encoding='utf-8', low_memory=False, header=0,)
    else:
        print('Raw input file paths must either be .csv or .txt files.')

    # increase component index to remain unique for multiple data frames
    this_data['Sample Index'] = this_data['Sample Index'] + sample_index
    sample_index += max(this_data['Sample Index'])
    check = this_data['Sample Index'].value_counts()

    # make sure each sample name ends with Ext for extended method and with Core for core method
    if file[-12:-4] == 'extended':
        mask_names = this_data['Sample Name'].str.endswith('Ext')
        this_data['Sample Name'][~mask_names] = [compound + ' Ext' for compound in this_data['Sample Name'][~mask_names].to_list()]

        # checks if extended data has 107 channels
        for index, elem in check[(check != 108)].items():
            print(f'The sample with Index {index} has {elem} channels.')

    elif file[-8:-4] == 'core':
        mask_names = this_data['Sample Name'].str.endswith('Core')
        this_data['Sample Name'][~mask_names] = [compound + ' Core' for compound in this_data['Sample Name'][~mask_names].to_list()]

        # checks if extended data has 107 channels
        for index, elem in check[(check != 142)].items():
            print(f'The sample with Index {index} has {elem} channels.')
    else:
        print('If you combine core method and extended method make sure your input file names end with _core and _extended respectively.')

    # append actual dataframe in list (this_data) to huge dataframe (data)
    if data.empty:
        data = this_data[columns_considered]  # initialize data in first step (when data is empty)
    else:
        data = pd.concat([data, this_data[columns_considered]], ignore_index=True)  # append to data

# Correct channel names in original data (all of the TOF channels are labelled by _TOF MS, only 2 of them are labeled by only _TOF)
mask_names = data['Component Name'].str.endswith('_TOF')
data['Component Name'][mask_names] = [compound + ' MS' for compound in data['Component Name'][mask_names].to_list()]

# some have an underscore between TOF and MS, this is removed
mask_names = data['Component Name'].str.endswith('_TOF_MS')
data['Component Name'][mask_names] = [compound[:-3] + ' MS' for compound in data['Component Name'][mask_names].to_list()]

First make sure that each sample of the core method is assigned correctly to a corresponding sample of the extended method.\
If duplicates of samples with the exact same name are available the assignment works just by the order or the sample index.

Then assign each PFAS component detection channels of precursor masses (_TOF MS) to the channel detecting its fragements (default).
Currently, some PFAS components are only detected in the _TOF MS channel, then the corresponding fragments are set to NaN.

In [ ]:
# make sample index equal if one sample exist as both "Core Method" and "Extended Method"
sample_names = data['Sample Name'].unique()

# combine indices of Core Method and Extended Method if both are available
detect = 0
index_mapper = {}
for sample_name in sample_names:
    if sample_name[-3:] == 'Ext':
        if sample_name[:-3] + 'Core' in sample_names:
            sample_indices_core = data.loc[data['Sample Name'].isin([sample_name[:-3] + 'Core']), 'Sample Index'].value_counts().index
            sample_indices_extended = data.loc[data['Sample Name'].isin([sample_name]), 'Sample Index'].value_counts().index
            if len(sample_indices_core) != len(sample_indices_extended):
                print(f'At least one of the sample {sample_name} has no related Core Method. Find out if this is a problem.')
            for index_core, index_extended in zip(sample_indices_core, sample_indices_extended):
                data.loc[data['Sample Index'].isin([index_core, index_extended]), 'Sample Index'] = index_core
                index_mapper[index_core] = sample_name[:-3]
                detect+=1
        else:
            print(f'The sample {sample_name} has no related Core Method. Find out if this is a problem.')
    elif sample_name[-4:] == 'Core':
        continue
    else:
        print(f'Make sure your input data name ends either with _core or with _extended.')

In [ ]:
# Get the order of components and split it to default channel (MS/MS) and TOF channel.
# If either channel is not available a copy of the other one is used respectively.
first_sample_id = data['Sample Index'].value_counts().index[0]  # index of first sample
components_filtered = data.loc[data['Sample Index'] == first_sample_id, 'Component Name']  # channel names of first sample
components_sorted = components_filtered[~components_filtered.str.contains(standard_identifiers)].to_list()  # channel names excluding IPS and IDA
ida_ips_sorted = components_filtered[components_filtered.str.contains(standard_identifiers)].to_list()  # channel names excluding IPS and IDA

# initialize and fill lists of sorted components
components_fragmented = []
components_precursor = []
skip_components = []
for component in components_sorted:
    if component in skip_components:
        continue
    if '_TOF MS' in component:
        components_precursor.append(component)
        skip_components.append(component)
        if component[:-7] in components_sorted:
            components_fragmented.append(component[:-7])
        else:
            components_fragmented.append(np.nan)
        skip_components.append(component[:-7])
    else:
        components_fragmented.append(component)
        skip_components.append(component)
        if component + '_TOF MS' in components_sorted:
            components_precursor.append(component + '_TOF MS')
        else:
            components_precursor.append(np.nan)
        skip_components.append(component + '_TOF MS')

print(components_fragmented, len(components_fragmented))
print(components_precursor, len(components_precursor))

# initialize and fill lists of sorted internal standards
ida_ips_fragmented = []
ida_ips_precursor = []
skip_standards = []
for standard in ida_ips_sorted:
    if standard in skip_standards:
        continue
    if '_TOF MS' not in standard:
        ida_ips_fragmented.append(standard)
        skip_standards.append(standard)
        if standard[4:] + '_TOF MS' in ida_ips_sorted:
            ida_ips_precursor.append(standard[4:] + '_TOF MS')
            skip_standards.append(standard[4:] + '_TOF MS')
        else:
            ida_ips_precursor.append(np.nan)
    else:
        if 'IDA-' + standard[:-7] in ida_ips_sorted:
            ida_ips_precursor.append(standard)
            skip_standards.append(standard)
            if standard[4:] + '_TOF MS' in ida_ips_sorted:
                ida_ips_fragmented.append('IDA-' + standard[:-7])
                skip_standards.append('IDA-' + standard[:-7])
            else:
                ida_ips_fragmented.append(np.nan)
        elif 'IPS-' + standard[:-7] in ida_ips_sorted:
            ida_ips_precursor.append(standard)
            skip_standards.append(standard)
            if standard[4:] + '_TOF MS' in ida_ips_sorted:
                ida_ips_fragmented.append('IPS-' + standard[:-7])
                skip_standards.append('IPS-' + standard[:-7])
            else:
                ida_ips_fragmented.append(np.nan)
        else:
            print(f'The standard: {standard} has no corresponding IDA or IPS in the default MS channel. It is ignored in the following calculations.')

In [ ]:
# select only calibration data
data = data[(data['Sample Type'] == 'Standard')]

# extract IDA data
data_ida = data.loc[(
    (data['Component Name'].str.contains(standard_identifiers)) & ((data['Sample Index']==data['Sample Index'][0]))
    ), ['Component Name', 'Component Group Name']]

# get rid of IDA and IPS
data_components = data[~data['Component Name'].str.contains(standard_identifiers)]

In [ ]:
# get retention times and deviations
retention_time = data[[
    'Component Name',  'Expected RT'
]]

retention_time['retention_time'] = retention_time['Expected RT'] * 60
retention_time_final = retention_time.groupby('Component Name', dropna=False).mean()['retention_time'].round(decimals=0)

retention_time_check = retention_time.groupby('Component Name', dropna=False).std()['retention_time']
retention_time_check.dropna(inplace=True)
if any(retention_time_check > 0):
    print(f'Retention time seems to be variable.')

In [ ]:
# get precursor mass and fragment mass
masses = data[[
    'Component Name',  'Precursor Mass', 'Fragment Mass'
]]
masses_check = masses.groupby('Component Name').std()
precursor_check = not all(masses_check['Precursor Mass'].isin([0, np.NaN]) == True)
fragment_check = not all(masses_check['Fragment Mass'].isin([0, np.NaN]) == True)
if precursor_check or fragment_check:
    print('Masses seem to be variable, double check evaluation method')
masses = masses.groupby('Component Name', dropna=False).mean()

masses['m/z'] = masses['Precursor Mass']
masses.loc[~masses.index.str.contains('_TOF MS'), 'm/z'] = masses.loc[~masses.index.str.contains('_TOF MS'), 'Fragment Mass']
masses.drop(columns=['Precursor Mass', 'Fragment Mass'], inplace=True)

# merge all dataframes
channels = masses.merge(retention_time_final, left_index=True, right_index=True)

In [ ]:
# Load idl values from IDL_2024 file
idl_2024 = pd.read_csv(idl_2024_filepath, index_col=0, low_memory=False, nrows=1)
idl_2024 = idl_2024.drop(columns=['Unnamed: 55'])

In [ ]:
channel_data = pd.DataFrame(columns=(
    'internal_name',  'precursor_channel_name', 'precursor_m_z_ratio', 'precursor_retention_time',
    'fragment_channel_name', 'fragment_m_z_ratio', 'fragment_retention_time',
    )
    )
index = 0
mschemicals = {}
# loop over component precursors and fragements, collect molecular masses and times and 
for (component_precursor, component_fragment) in zip(components_precursor + ida_ips_precursor, components_fragmented + ida_ips_fragmented):
    
    if component_precursor is np.NaN:
        precursor_row = pd.Series(data={
            'retention_time': np.NaN,
            'm/z': np.NaN,
        })
        component_precursor = np.NaN
    else:
        internal_name = component_precursor[:-7]
        precursor_row = channels.loc[component_precursor, :]

    if component_fragment is np.NaN:
        fragment_row = pd.Series(data={
            'retention_time': np.NaN,
            'm/z': np.NaN,
        })
        component_fragment = np.NaN
    else:
        internal_name = component_fragment
        fragment_row = channels.loc[component_fragment, :]

    channel_data.loc[index, 'internal_name'] = internal_name
    channel_data.loc[index, 'precursor_channel_name'] = component_precursor
    channel_data.loc[index, 'precursor_m_z_ratio'] = precursor_row['m/z']
    channel_data.loc[index, 'precursor_retention_time'] = precursor_row['retention_time']
    channel_data.loc[index, 'fragment_channel_name'] = component_fragment
    channel_data.loc[index, 'fragment_m_z_ratio'] = fragment_row['m/z']
    channel_data.loc[index, 'fragment_retention_time'] = fragment_row['retention_time']
    mschemicals[internal_name] = pyda.MSChemical(
        precursor_channel_name=component_precursor, precursor_m_z_ratio=precursor_row['m/z'],
        precursor_retention_time=precursor_row['retention_time'],
        fragment_channel_name=component, fragment_m_z_ratio=fragment_row['m/z'],
        fragment_retention_time=fragment_row['retention_time'],
    )
    index += 1

channel_data.to_csv('outputs//chemicals//channels.csv')


In [ ]:
# make PFAS data frame and add IDA IPS on a component basis
basics = data_components[(data_components['Sample Index']==data_components['Sample Index'][0])]

components_data = pd.DataFrame(columns=(
    'internal_name', 'name', 'pubchem', 'cas', 'smiles',
    'ida', 'ips', 'instrumentation_detection_limit',
    )
    )

# read in references from excel sheet and add them to information
references = pd.read_excel('PFAS_STEEP_identifiers.xlsx')

# create dictionary mapping indices to component names and abbreviations
abbreviations_dict = {}
for index in references.index:
    abbreviation = [references.loc[index, 'Abbreviation']]
    synonyms = references.loc[index, 'Synonyms']
    if synonyms is not np.nan:
        for synonym in str(synonyms).split(','):
            abbreviation.append(synonym)
    abbreviations_dict[index] = abbreviation

# initialize variable list of chemicals
list_of_chemicals = []

# loop over component precursors, collect all the information for each precursor and put it to dataframe components_data
for (component_index, component_precursor) in enumerate(components_precursor):
    # get name of fragemented component, the corresponding index in the reference excel by abbreviations and synonyms and extract relevant row from the excel
    for reference_index, abbreviations in abbreviations_dict.items():
        if component_precursor[:-7] in abbreviations:
            reference_line = references.iloc[reference_index, :]
            break
    
    # fill empty data in name is not available
    if reference_line.empty:
        print(f'There is no data for {component_precursor[:-7]} available.')
        reference_line = pd.Series(data={
            'Compound name': np.NaN,
            'CAS': np.NaN,
            'SMILES': np.NaN,
        })
    chemical = mschemicals[component_precursor[:-7]]

    component_fragmented = components_fragmented[component_index]
    # get ida if available
    if component_fragmented is not np.NaN:
        ida_name = basics.loc[basics['Component Name'].isin([component_fragmented]), 'IS Name'].values[0]
        ida = mschemicals[ida_name]
    else:
        ida_name = np.NaN
        ida = np.NaN
    
    # get ips if available
    if ida_name in [np.NaN, 'nan']:
        ips_name = np.NaN
        ips = np.NaN
    else:
        ips_name = data_ida.loc[data_ida['Component Name'].isin([ida_name]), 'Component Group Name'].values[0]
        ips = mschemicals[ips_name]

    if component_precursor[:-7] in idl_2024.columns:
        idl = idl_2024.loc['IDL', component_precursor[:-7]]
    else:
        idl = np.NaN

    components_data.loc[component_index, 'internal_name'] = component_precursor[:-7]
    components_data.loc[component_index, 'name'] = reference_line.loc['Compound name']

    pubchemlink = (reference_line.loc['PubChem link'])
    cas = reference_line.loc['CAS']
    smiles = reference_line.loc['SMILES']
    if not pubchemlink is np.NaN:
        pubchemlink = pubchemlink[42:]

    components_data.loc[component_index, 'pubchem'] = pubchemlink
    components_data.loc[component_index, 'cas'] = cas
    components_data.loc[component_index, 'smiles'] = smiles
    components_data.loc[component_index, 'ida'] = ida_name
    components_data.loc[component_index, 'ips'] = ips_name
    components_data.loc[component_index, 'instrumentation_detection_limit'] = idl
    list_of_chemicals.append(pyda.ChemicalSubstance(
        internal_name=component_precursor[:-7], cas=cas, smiles=smiles,
        name=reference_line.loc['Compound name'], pubchem=pubchemlink,
        chemical=chemical , ida=ida, ips=ips, instrumentation_detection_limit=idl,
    ))

root = pyda.ListOfChemicals(pfas=list_of_chemicals)

components_data.to_csv('outputs//chemicals//components.csv')

# Convert and validate

In [ ]:
# Convert data to json
yaml_data = yaml_dumper.dumps(root)
with open("outputs/chemicals/data.yaml", "w") as f:
    f.write(yaml_data)

# Validate
validate_data_to_schema(yamlschema="chemicals.yaml", yamldata="outputs/chemicals/data.yaml")

# Convert data to .csv
convert_data_to_csv(
    yamlschema="chemicals.yaml", yamldata="outputs/chemicals/data.yaml", csvdata='outputs/chemicals/data.csv',
    )

# Generate Documentation

In [ ]:
# generate documentation
generate_schema_documentation(
    yamlschema='chemicals.yaml', directory='outputs/chemicals', example_directory='outputs/chemicals/data.yaml')